# Emotional AI Chatbot Using Dorner's Psi Theory

This notebook explores the development of an emotional AI chatbot by integrating Dorner's Psi Theory for understanding and simulating emotional responses. It dynamically calculates and displays sadness and anger levels during a conversation, adapting its dialogue accordingly.

### Step 1:Importing Libraries

In [6]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from flask import Flask, render_template, request, jsonify

### Step 2: Define Emotional Model

#### Dorner's Psi Theory Parameters

In [7]:
# Valence: Spectrum of attraction vs. aversion (1-7).
# Arousal: Readiness for action (1-7).
# Selection Threshold: Ease of intention shifting (1-7).
# Resolution: Perceptual accuracy (1-7).
# Goal Directedness: Stability of motives (1-7).

def calculate_emotional_state(valence, arousal, selection_threshold, resolution, goal_directedness):
    """
    Calculate the anger and sadness levels based on Dorner's Psi Theory parameters.
    
    Parameters:
        valence (float): Spectrum of attraction vs. aversion (1-7).
        arousal (float): Readiness for action (1-7).
        selection_threshold (float): Ease of intention shifting (1-7).
        resolution (float): Perceptual accuracy (1-7).
        goal_directedness (float): Stability of motives (1-7).
    
    Returns:
        dict: Calculated anger and sadness levels.
    """
    anger = max(0, (arousal * (7 - valence) * goal_directedness) / 35)
    sadness = max(0, ((7 - arousal) * (7 - goal_directedness)) / 49)
    return {
        "anger": round(min(anger, 5), 2),
        "sadness": round(min(sadness, 5), 2)
    }

# Test the function with example values
example_emotions = calculate_emotional_state(4, 6, 5, 4, 3)
print("Example Emotion Levels:", example_emotions)

Example Emotion Levels: {'anger': 1.54, 'sadness': 0.08}


### Step 4: Set Up the Chatbot
### Define Chatbot Logic Using OpenRouter and Google Gemini Model

In [8]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain_core.runnables import Runnable
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Initialize LangChain's memory for conversation context
memory = ConversationBufferMemory()

# Configuration for OpenRouter API
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL = os.getenv("OPENROUTER_BASE_URL")

def initialize_chatbot():
    """Initialize the chatbot using OpenRouter with Google Gemini model."""
    client = OpenAI(
        base_url=OPENROUTER_BASE_URL,
        api_key=OPENROUTER_API_KEY,
    )

    class GeminiWrapper(Runnable):
        def invoke(self, prompt, config=None, **kwargs):
            completion = client.chat.completions.create(
                model="google/gemini-exp-1121:free",
                messages=[{"role": "user", "content": prompt}]
            )
            return completion.choices[0].message.content

    chain = ConversationChain(llm=GeminiWrapper(), memory=memory, verbose=True)
    return chain

chatbot = initialize_chatbot()